In [7]:
from bs4 import BeautifulSoup
import requests
import json
from urllib.parse import urlparse
import pandas as pd


In [50]:
from google.colab import files
upload = files.upload()


Saving data_clean_text_analisis.csv.csv to data_clean_text_analisis.csv (1).csv


In [51]:
df = pd.read_csv('data_clean_text_analisis.csv.csv')
df

,URLs,Headline,Body,Label,Body_01,Body_02,Number of spell mistakes,Body_03,Body_04,Polarity,Subjectivity
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\r\nOn Sunday morn...,1,image copyright getty imageson sunday morning ...,image copyright getty images sunday morning do...,4,image copyright getty image unday morning dona...,image copyright getty image unday morning dona...,0.457409,0.422146
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...,london reuters last flag flying a comedy dram...,0,london reuter lat flag fly a comedy drama abou...,london reuter lat flag fly comedy drama vietna...,0.474333,0.460599
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...,the feud broke into public view last week when...,3,the feud break into public view lat week when ...,feud break public view lat week mr corker ay ...,0.512022,0.408650
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...,mexico city reuters egypt s chevron holdings l...,1,mexico city reuter egyptchevron holding limite...,mexico city reuter egyptchevron holding limite...,0.502063,0.282653
4,http://beforeitsnews.com/sports/2017/09/jetnat...,JetNation FanDuel League; Week 4,JetNation FanDuel League; Week 4\r\n% of reade...,0,jetnation fanduel league week of readers think...,detention fanduel league week of readers think...,3,detention fanduel league week of reader think ...,detention fanduel league week reader think thi...,0.425532,0.337500
...,...,...,...,...,...,...,...,...,...,...,...
2371,https://www.activistpost.com/2017/09/false-arr...,35 False Matches and 1 Erroneous Arrest As Pol...,By Nicholas West\r\nAll things biometric are s...,0,by nicholas westall things biometric are sweep...,by nicholas westall things biometric are sweep...,1,by nichola wetall thing biometric be weep acro...,nichola wetall thing biometric weep acro worl...,0.483309,0.308180
2372,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,vietnam is in great danger you must publish an...,vietnam is in great danger you must publish an...,0,vietnam be in great dangermut publih and tella...,vietnam great dangermut publih tellarmy govern...,0.936170,0.750000
2373,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\r\n% of readers think this sto...,0,trends to watch of readers think this story is...,trends to watch of readers think this story is...,8,trend to watch of reader think thi tory be fac...,trend watch reader think thi tory fact addtwo ...,0.630722,0.516919
2374,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr is soon to give a minute speech for ...,trump jr is soon to give a minute speech for ...,0,trump jr be oon to give a minute peech for of ...,trump jr oon give minute peech reader think th...,0.483559,0.303030


In [9]:
for i in range(2376):
    if('\n\n' in df['URLs'][i]):
        df['URLs'][i] = df['URLs'][i][0:-2]
    if('\r\n' in df['URLs'][i]):
        df['URLs'][i] = df['URLs'][i][0:-2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [16]:
def action_parser(link):
    
    req = requests.get(link)
    if req.status_code==200 or req.status_code==403:
      soup = BeautifulSoup(req.text, "lxml")
      head = soup.find('head')
      zagolovok = head.find('title').text
      textp = soup.find_all('div', attrs={'class': "article__text"})
      if (len(textp)) == 0:
          textp = soup.find_all('span', attrs={'class': 'intro'})
      if (len(textp)) == 0:
          text = soup.find_all('span', attrs={'class': 'intro'})
      if (len(textp)) == 0:
          textp = soup.find_all('p')
      data = soup.find_all('time')
      domain = urlparse(link).netloc
      if (int(domain.find("www.")) != -1):
        domain = domain[4:]
      if (domain =='cnn.com' and len(data)==0) :
        data = soup.find_all('p', attrs={'class': 'update-time'})
    

      if len(data) == 0:
          data = soup.find_all('span', attrs={'class': 'article__info-date-modified'})
      if len(data) == 0:
          data = soup.find_all('div', attrs={'class': 'article__info-date-modified'})

      if len(data) == 0:
        data = soup.find_all('span', attrs={'class': 'DateLine__date___12trWy'})
      if len(data) == 0:
        data = soup.find_all('font', attrs={'style': 'vertical-align: inherit;'})
      if len(data) == 0:
        data = soup.find_all('div', attrs={'class': 'css-19m31ns'})
      if len(data) == 0:
        data = soup.find_all('p', attrs={'class': 'update-time'}) 
      if len(data) == 0:
        data = soup.find_all('span', attrs={'class': 'update-time'})
      if len(data) == 0:
        data = soup.find_all('div', attrs={'class': 'update-time'})#http://www.cnn.com/2017/10/09/politics/melania-trump-ivana-trump-first-lady/index.html
      if len(data)==0:
        data = soup.find_all('p', attrs={'class': 'update-time'})
      if len(data) == 0:
        data = soup.find_all('p', attrs={'class': 'entry-meta'})
      if len(data) == 0:
        data = soup.find_all('div', attrs={'class': "post_date"}) 
      if (len(data) ==0 and domain=="activistpost.com"):
        # https://www.activistpost.com/2017/09/september-11-2001-questions-believe-official-story.html
        re = requests.get(link, headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'})
        if re.status_code==200:
          soup = BeautifulSoup(re.text, "lxml")
          head = soup.find('head')
          data = soup.find_all('span', attrs={'class': 'entry-meta-date updated'})
          zagolovok = head.find('title').text

      if len(data) == 0:
        data = soup.find_all('p', attrs={'class': 'entry-meta'})
      if len(data) == 0:
        data = soup.find_all('div', attrs={'class': "post_date"}) 
      if len(data)==0:
        r = requests.get('https://www.reuters.com/article/us-filmfestival-london-lastflagflying/linklaters-war-veteran-comedy-speaks-to-modern-america-says-star-idUSKBN1CD0X2',  headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'})
        if r.status_code==200:
          soupe = BeautifulSoup(r.text, "lxml")
          data= soupe.find_all('time', attrs={'class': 'TextLabel__text-label___3oCVw TextLabel__gray___1V4fk TextLabel__small-all-caps___2Z2RG ArticleHeader-date-Goy3y'})
      data1=data[0].text
      # print(zagolovok)

    

      # for i in range(len(textp)):
      #     print(textp[i].text)

      # if len(data) != 0:
      #     for i in range(len(data)):
      #         print(data[i].text)
      # else:
      #     print("дата публикации на новстоном ресусре не указана")

      # print(data.text)

    

      url = link.split("//")[-1].split("/")[0].split('?')[0]
      show = "https://input.payapi.io/v1/api/fraud/domain/age/" + domain

      r = requests.get(show, verify = False)
      soupp = BeautifulSoup(r.text, "lxml")

      if r.status_code == 200:

          age = soupp.find('body').text
          str = int(age.find("Reg"))
          end = int(age.find("}"))
          # print(age[str:(end - 1)])

          data = r.text
          jsonToPython = json.loads(data)

          list = [zagolovok, data1, age[str:(end - 1)], domain]
          return list


In [24]:
dann= []
for i in range(len(df)):
  llink=action_parser(df['URLs'][i])
  dann.append(llink)
  print(i)


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


1


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2369
2370


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2371
2372
2373
2374
2375


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [52]:
zag=[]
for i in range(len(dann)):
  if dann[i]!= None:
    zag.append(dann[i][0])
  else:
    zag.append('')

da=[]
for i in range(len(dann)):
  if dann[i]!= None:
    da.append(dann[i][1])
  else:
    da.append('')
ag=[]
for i in range(len(dann)):
  if dann[i]!= None:
    ag.append(dann[i][2])
  else:
    ag.append('')
dom=[]
for i in range(len(dann)):
  if dann[i]!= None:
    dom.append(dann[i][3])
  else:
    dom.append('')

In [56]:
df



,URLs,Headline,Body,Label,Body_01,Body_02,Number of spell mistakes,Body_03,Body_04,Polarity,Subjectivity,zagolovok,date,age,domain
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\r\nOn Sunday morn...,1,image copyright getty imageson sunday morning ...,image copyright getty images sunday morning do...,4,image copyright getty image unday morning dona...,image copyright getty image unday morning dona...,0.457409,0.422146,Four ways Bob Corker skewered Donald Trump - B...,9 October 2017,Reg. Date: 1989-07-15,bbc.com
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...,london reuters last flag flying a comedy dram...,0,london reuter lat flag fly a comedy drama abou...,london reuter lat flag fly comedy drama vietna...,0.474333,0.460599,Linklater's war veteran comedy speaks to moder...,,Reg. Date: 1993-06-03,reuters.com
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...,the feud broke into public view last week when...,3,the feud break into public view lat week when ...,feud break public view lat week mr corker ay ...,0.512022,0.408650,Trump’s Fight With Corker Jeopardizes His Legi...,"Oct. 9, 2017",Reg. Date: 1994-01-18,nytimes.com
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...,mexico city reuters egypt s chevron holdings l...,1,mexico city reuter egyptchevron holding limite...,mexico city reuter egyptchevron holding limite...,0.502063,0.282653,Egypt's Cheiron wins tie-up with Pemex for Mex...,,Reg. Date: 1993-06-03,reuters.com
4,http://beforeitsnews.com/sports/2017/09/jetnat...,JetNation FanDuel League; Week 4,JetNation FanDuel League; Week 4\r\n% of reade...,0,jetnation fanduel league week of readers think...,detention fanduel league week of readers think...,3,detention fanduel league week of reader think ...,detention fanduel league week reader think thi...,0.425532,0.337500,JetNation FanDuel League; Week 4 | Sports | Be...,"\nWednesday, September 27, 2017 14:42",Reg. Date: 2007-09-15,beforeitsnews.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,https://www.activistpost.com/2017/09/false-arr...,35 False Matches and 1 Erroneous Arrest As Pol...,By Nicholas West\r\nAll things biometric are s...,0,by nicholas westall things biometric are sweep...,by nicholas westall things biometric are sweep...,1,by nichola wetall thing biometric be weep acro...,nichola wetall thing biometric weep acro worl...,0.483309,0.308180,35 False Matches and 1 Erroneous Arrest As Pol...,"September 7, 2017",Reg. Date: 2010-04-23,activistpost.com
2372,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,vietnam is in great danger you must publish an...,vietnam is in great danger you must publish an...,0,vietnam be in great dangermut publih and tella...,vietnam great dangermut publih tellarmy govern...,0.936170,0.750000,,,,
2373,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\r\n% of readers think this sto...,0,trends to watch of readers think this story is...,trends to watch of readers think this story is...,8,trend to watch of reader think thi tory be fac...,trend watch reader think thi tory fact addtwo ...,0.630722,0.516919,,,,
2374,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr is soon to give a minute speech for ...,trump jr is soon to give a minute speech for 

In [64]:
df_parser=df[['zagolovok', 'date', 'age', 'domain']]
#df_parser['date']=df['date']
#df_parser['age']=df['age']
#df_parser['domain']=df['domain']

In [65]:
df_parser

,zagolovok,date,age,domain
0,Four ways Bob Corker skewered Donald Trump - B...,9 October 2017,Reg. Date: 1989-07-15,bbc.com
1,Linklater's war veteran comedy speaks to moder...,,Reg. Date: 1993-06-03,reuters.com
2,Trump’s Fight With Corker Jeopardizes His Legi...,"Oct. 9, 2017",Reg. Date: 1994-01-18,nytimes.com
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,,Reg. Date: 1993-06-03,reuters.com
4,JetNation FanDuel League; Week 4 | Sports | Be...,"\nWednesday, September 27, 2017 14:42",Reg. Date: 2007-09-15,beforeitsnews.com
...,...,...,...,...
2371,35 False Matches and 1 Erroneous Arrest As Pol...,"September 7, 2017",Reg. Date: 2010-04-23,activistpost.com
2372,,,,
2373,,,,
2374,,,,


In [66]:
df_parser.to_csv('data_after_parsing', index=False)
files.download('data_after_parsing')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>